# Project Assignment

* [link to d3 home page](https://titanbender.github.io) 
* [link to git repo](https://github.com/titanbender/titanbender.github.io) 

In [1]:
from IPython.display import VimeoVideo
VimeoVideo("211653855",width=640, height=360)

In [3]:
# Used libraries
import json
import numpy as np
import pandas as pd
from __future__ import division
import matplotlib.pyplot as plt
import collections 
from sklearn.cluster import KMeans
import datetime as dt
import geoplotlib
from geoplotlib.utils import BoundingBox

## Defined functions

In [4]:
# Create JSON function 
def createJSON(my_dict, file_name):
    with open(file_name, 'w') as outfile:
        json.dump(my_dict, outfile)
        outfile.close()

## Load weatherdata

In [5]:
# load in the new york weather data
w_DF = pd.read_csv("data/939937.csv", low_memory = False)

## Data manipulation

In [4]:
## Data manipulation
# Load motorvehicle accident data
raw_DF = pd.read_csv("data/raw_data.csv", low_memory = False)

# Getting year
year = raw_DF["DATE"].apply(lambda x: x.split('/'))
raw_DF["YEAR"] = year.apply(lambda x: int(x[2]))

# Getting hour
hour = raw_DF["TIME"].apply(lambda x: x.split(':'))
raw_DF["HOUR"] = hour.apply(lambda x: int(x[0]))
raw_DF["TIME_DEC"] = hour.apply(lambda x: float(int(x[0]) + int(x[1])/60) )

# String values for month
month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
              'August', 'September', 'October', 'November', 'December']

# Getting month
month = raw_DF["DATE"].apply(lambda x: x.split('/'))
raw_DF["MONTH"] = month.apply(lambda x: month_list[int(x[0])-1])

# remove unused columns
raw_DF = raw_DF.drop(['ZIP CODE',
                          'LOCATION',
                          'UNIQUE KEY',
                          'ON STREET NAME',
                          'CROSS STREET NAME',
                          'OFF STREET NAME'], axis = 1)
# raw_DF.head()

raw_DF.shape

(601313, 27)

## Pratical data information

In [5]:
#list(raw_DF)

In [6]:
pra_info = {}
# First day of observations
pra_info['first obs'] = min(raw_DF["DATE"])
# Last day of observations
pra_info['last obs'] = max(raw_DF["DATE"])
# 
# pra_info['unique year'] = list(set(raw_DF['YEAR']))
pra_info['unique year'] = [2013,2014,2015,2016]
#
pra_info['N'] = {}
tmp = []
for ii in pra_info['unique year']:
    pra_info['N'][ii] = len(raw_DF[raw_DF['YEAR'] == ii])
    tmp.append(pra_info['N'][ii])
# sum tot
pra_info['N']['tot'] = sum(tmp)

pra_info['unique borough'] = list(set(raw_DF['BOROUGH']))
pra_info['borough'] = {}
for ii in pra_info['unique borough']:
    pra_info['borough'][ii] = {}
    tmp = []
    for jj in pra_info['unique year']:
        pra_info['borough'][ii][jj] = len(raw_DF[(raw_DF['YEAR'] == jj) & (raw_DF['BOROUGH'] == ii)])
        tmp.append(pra_info['borough'][ii][jj])   
    # sum tot
    pra_info['borough'][ii]['tot'] = sum(tmp)
    
# obs per month
pra_info['unique months'] = list(set(raw_DF['MONTH']))
pra_info['N month'] = {}
for ii in pra_info['unique months']:
    pra_info['N month'][ii] = {}
    tmp = []
    for jj in pra_info['unique year']:
        pra_info['N month'][ii][jj] = len(raw_DF[(raw_DF['YEAR'] == jj) & (raw_DF['MONTH'] == ii)])
        tmp.append(pra_info['N month'][ii][jj])
        
    # sum tot
    pra_info['N month'][ii]['tot'] = sum(tmp)
    
# ratio
pra_info['ratio'] = {}
for ii in pra_info['unique borough']:
    pra_info['ratio'][ii] = {}
    for jj in pra_info['unique year']+['tot']:
        pra_info['ratio'][ii][jj] = pra_info['borough'][ii][jj] / pra_info['N'][jj] 
        
# ratio month
pra_info['ratio month'] = {}
for ii in pra_info['unique months']:
    pra_info['ratio month'][ii] = {}
    for jj in pra_info['unique year']:
        pra_info['ratio month'][ii][jj] = pra_info['N month'][ii][jj] / pra_info['N'][jj]
        
# obs per day
pra_info['unique days'] = list(set(raw_DF['DATE']))
pra_info['No. days'] = {}
for ii in pra_info['unique days']:
    pra_info['No. days'][ii] = len(raw_DF[raw_DF['DATE'] == ii])
    
# avg
# counting
tmp = [int(pra_info['No. days'].keys()[ii].split('/')[2]) for ii in range(0, len(pra_info['No. days']))]
tmp_days = collections.Counter(tmp)
tmp_days['tot'] = len(pra_info['No. days'])

pra_info['avg day'] = {}
for ii in pra_info['unique year']+['tot']:
        pra_info['avg day'][ii] =  pra_info['N'][ii] / tmp_days[ii]
        
pra_info['avg year'] = {}
for ii in pra_info['unique year']:
        pra_info['avg year'][ii] =  pra_info['N'][ii] / len(pra_info['unique year'])


In [7]:
# Pratical data information prints
print "First day of observations:", min(raw_DF["DATE"])
print "Last day of observations:", max(raw_DF["DATE"])
print "Total number of obs. in mentioned period:" , len(raw_DF['DATE'])
print ""
print "Observations per month:"
print pra_info['N month']
print ""
print "Accidents per district:"
print pra_info['borough']
print ""
print "Geographic area of each district:"
print "To be done.... "
print ""
print "Ratio between geographic area per district and number of accidents"
print pra_info['ratio']
print ""
print "Ratio between accidents and months"
print pra_info['ratio month']
print ""
print "Types of accidents"
print "To be done.... "
print ""
print "Average accidents per year:"
print pra_info['avg year']
print ""
print "Average accidents per day:"
print pra_info['avg day'] #pra_info['N']['tot'] / len(pra_info['No. days'])
print ""
print "Day with most accidents"
print sorted(pra_info['No. days'].items(), key=lambda x: x[1], reverse=True)[0]
print ""
print "Day with least accidents"
print sorted(pra_info['No. days'].items(), key=lambda x: x[1], reverse=False)[0]
print ""
print "Top 10 days with most accidents:" 
print sorted(pra_info['No. days'].items(), key=lambda x: x[1], reverse=True)[0:10]
print ""
print "Top 10 days with least accidents:" 
print sorted(pra_info['No. days'].items(), key=lambda x: x[1], reverse=False)[0:10]


First day of observations: 01/01/2013
Last day of observations: 12/31/2016
Total number of obs. in mentioned period: 601313

Observations per month:
{'February': {2016: 12170, 'tot': 46477, 2013: 11246, 2014: 12142, 2015: 10919}, 'October': {2016: 10435, 'tot': 52628, 2013: 13417, 2014: 13773, 2015: 15003}, 'March': {2016: 13819, 'tot': 52484, 2013: 12809, 2014: 12127, 2015: 13729}, 'August': {2016: 565, 'tot': 40289, 2013: 12654, 2014: 12861, 2015: 14209}, 'September': {2016: 1157, 'tot': 41507, 2013: 12858, 2014: 13496, 2015: 13996}, 'December': {2016: 11421, 'tot': 51799, 2013: 13197, 2014: 13023, 2015: 14158}, 'June': {2016: 12277, 'tot': 54375, 2013: 14045, 2014: 13966, 2015: 14087}, 'April': {2016: 13765, 'tot': 51490, 2013: 12712, 2014: 12413, 2015: 12600}, 'May': {2016: 13880, 'tot': 56432, 2013: 14334, 2014: 13708, 2015: 14510}, 'January': {2016: 13945, 'tot': 51049, 2013: 11979, 2014: 12713, 2015: 12412}, 'November': {2016: 11372, 'tot': 51315, 2013: 13427, 2014: 12884, 2015:

## 1. Motivation
- What is your dataset?
- Why did you choose this/these particular dataset(s)?
- What was your goal for the end user's experience?

## 2. Basic stats. Let's understand the dataset better
- Write about your choices in data cleaning and preprocessing
- Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)

## 3. Theory. Which theoretical tools did you use?
- Describe which machine learning tools you use and why the tools you've chosen are right for the problem you're solving.
- Talk about your model selection. How did you split the data in to test/training. Did you use cross validation?
- Explain the model performance. How did you measure it? Are your results what you expected?

## 4. Visualizations
- Explain the visualizations you've chosen.
- Why are they right for the story you want to tell?

In [8]:
# Time Series
pra_info['ts_json'] = []
            
# loop for each day
for dd in pra_info['unique days']:
    # init dict
    tmp = {}
    
    # add date
    tmp['date'] = dd
    
    # sum number of tot_acc pr. day
    tmp['tot_acc'] = len(raw_DF[raw_DF['DATE'] == dd])
    
    # loop for each borough
    for bb in pra_info['unique borough']:
        
        # find indices for unique day and unique borough
        tmp_idx = list(raw_DF[(raw_DF['DATE'] == dd) & (raw_DF['BOROUGH'] == bb)].index)
        #
        tmp[bb + '_ped_inj'] = np.sum(raw_DF['NUMBER OF PEDESTRIANS INJURED'].iloc[tmp_idx])
        tmp[bb + '_ped_kill'] = np.sum(raw_DF['NUMBER OF PEDESTRIANS KILLED'].iloc[tmp_idx])
        tmp[bb + '_ped_tot'] = (tmp[bb + '_ped_inj'] + 
                                tmp[bb + '_ped_kill'])
        
        tmp[bb + '_cyc_inj'] = np.sum(raw_DF['NUMBER OF CYCLIST INJURED'].iloc[tmp_idx])
        tmp[bb + '_cyc_kill'] = np.sum(raw_DF['NUMBER OF CYCLIST KILLED'].iloc[tmp_idx])
        tmp[bb + '_cyc_tot'] = (tmp[bb + '_cyc_inj'] + 
                                tmp[bb + '_cyc_kill'])
        
        tmp[bb + '_mot_inj'] = np.sum(raw_DF['NUMBER OF MOTORIST INJURED'].iloc[tmp_idx])
        tmp[bb + '_mot_kill'] = np.sum(raw_DF['NUMBER OF MOTORIST KILLED'].iloc[tmp_idx])
        tmp[bb + '_mot_tot'] = (tmp[bb + '_mot_inj'] + 
                                tmp[bb + '_mot_kill'])
        
        tmp[bb + '_tot_inj'] = np.sum(raw_DF['NUMBER OF PERSONS INJURED'].iloc[tmp_idx])
        tmp[bb + '_tot_kill'] = np.sum(raw_DF['NUMBER OF PERSONS KILLED'].iloc[tmp_idx])
        tmp[bb + '_tot_tot'] = (tmp[bb + '_tot_inj'] + 
                                tmp[bb + '_tot_kill'])
    # append day and info to the json list
    pra_info['ts_json'].append(tmp)

# create json
tmp = pra_info['ts_json']
tmp.sort(key=lambda x: dt.datetime.strptime(x['date'], '%m/%d/%Y'))
createJSON(tmp,'data/ts_all.json')

In [23]:
# KNN structure 
tmp = []

K = range(2,5)

IN_col_d_tree = ['LATITUDE','LONGITUDE',]


Class = [
    'NUMBER OF PERSONS INJURED',
    'NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED',
    'NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED',
    'NUMBER OF CYCLIST KILLED',
    'NUMBER OF MOTORIST INJURED',
    'NUMBER OF MOTORIST KILLED']



#tmp_1 = list(raw_DF['NUMBER OF PERSONS INJURED'])
#tmp_2 = list(raw_DF['NUMBER OF PERSONS KILLED'])

# Getting year
#tmp = raw_DF['NUMBER OF PERSONS INJURED'].apply(lambda x: print x)
#raw_DF["YEAR"] = year.apply(lambda x: int(x[2]))


#tmp = raw_DF[['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']]

#injury = []

#for ee in range (0, len(raw_DF)):
#    if raw_DF['NUMBER OF PERSONS INJURED'][ee] > 0:
#        injury.append(True)
#    elif raw_DF['NUMBER OF PERSONS KILLED'][ee] > 0:
#        injury.append(True)
#    else:
#        injury.append(False)

#raw_DF['INJURED_KILLED'] = injury


In [10]:
# Decision tree structure

IN_col_d_tree = [
    'DATE',
    'LATITUDE',
    'LONGITUDE',
    'TIME_DEC',
    'MONTH',
    # below maybe
    'CONTRIBUTING FACTOR VEHICLE 1',
    'CONTRIBUTING FACTOR VEHICLE 2',
    'CONTRIBUTING FACTOR VEHICLE 3',
    'CONTRIBUTING FACTOR VEHICLE 4',
    'CONTRIBUTING FACTOR VEHICLE 5',
    'VEHICLE TYPE CODE 1',
    'VEHICLE TYPE CODE 2',
    'VEHICLE TYPE CODE 3',
    'VEHICLE TYPE CODE 4',
    'VEHICLE TYPE CODE 5']


OUT_col_d_tree = [
    'NUMBER OF PERSONS INJURED',
    'NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED',
    'NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED',
    'NUMBER OF CYCLIST KILLED',
    'NUMBER OF MOTORIST INJURED',
    'NUMBER OF MOTORIST KILLED']

In [20]:
list(raw_DF)

['DATE',
 'TIME',
 'BOROUGH',
 'LATITUDE',
 'LONGITUDE',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'CONTRIBUTING FACTOR VEHICLE 1',
 'CONTRIBUTING FACTOR VEHICLE 2',
 'CONTRIBUTING FACTOR VEHICLE 3',
 'CONTRIBUTING FACTOR VEHICLE 4',
 'CONTRIBUTING FACTOR VEHICLE 5',
 'VEHICLE TYPE CODE 1',
 'VEHICLE TYPE CODE 2',
 'VEHICLE TYPE CODE 3',
 'VEHICLE TYPE CODE 4',
 'VEHICLE TYPE CODE 5',
 'YEAR',
 'HOUR',
 'TIME_DEC',
 'MONTH',
 'INJURED_KILLED']

In [11]:
# Heat Map
import gmplot
gmap = gmplot.GoogleMapPlotter(np.mean(raw_DF["LATITUDE"]), np.mean(raw_DF["LONGITUDE"]), 10.5)

#gmap.plot(track['lat'], track['lon'], 'red', edge_width = 5)
#gmap.scatter(more_lats, more_lngs, '#3B0B39', size=40, marker=False)
#gmap.scatter(track['lat'], track['lon'], 'k', marker=True)
gmap.heatmap(list(raw_DF["LATITUDE"])[0:100000], list(raw_DF["LONGITUDE"])[0:100000], threshold=5, radius=5)

gmap.draw("data/heatmap[0-100000obs].html")

In [12]:
geo_data = {}
geo_data['lat'] = list(raw_DF["LATITUDE"])
geo_data['lon'] = list(raw_DF["LONGITUDE"])



#geoplotlib.kde(geo_data, cmap = 'jet', bw = 5, cut_below=1e-4)
geoplotlib.kde(geo_data, bw = 2, cut_below=1e-4)

bbox = BoundingBox(north = np.mean(raw_DF["LATITUDE"]) + 0.232,
                   west = np.mean(raw_DF["LONGITUDE"]) - 0.232,
                   south = np.mean(raw_DF["LATITUDE"]) - 0.232,
                   east = np.mean(raw_DF["LONGITUDE"]) + 0.232)

geoplotlib.set_bbox(bbox)
#geoplotlib.tiles_provider('watercolor')
#geoplotlib.inline()
geoplotlib.savefig('data/heatmap')

('smallest non-zero count', 4.4777898101688105e-09)
('max count:', 135.41750097693577)


In [13]:
# Create geojson
def parseJson(year,file_name):
    # number of obs. 
    tmp_DF = raw_DF[raw_DF["YEAR"] == year]
    #
    tmp_lat = list(tmp_DF["LATITUDE"])
    tmp_lon = list(tmp_DF["LONGITUDE"])
    tmp_hour = list(tmp_DF["HOUR"])
    tmp_borough = list(tmp_DF["BOROUGH"])
    # 
    N_tot = len(tmp_lat)
    
    # output
    out_dict = [{'lat': tmp_lat[ii],
                 'lon': tmp_lon[ii],
                 'r': 1,
                 #'borough': tmp_borough[ii],
                 #'hour': tmp_hour[ii],
                 #'class': "all"
                } for ii in range(0,N_tot)]
    #
    createJSON(out_dict, file_name)    

parseJson(2013, 'data/geo_plot_2013.json')
parseJson(2014, 'data/geo_plot_2014.json')
parseJson(2015, 'data/geo_plot_2015.json')
parseJson(2016, 'data/geo_plot_2016.json')

## 5. Discussion. Think critically about your creation
- What went well?,
- What is still missing? What could be improved?, Why?

## Some additional notes:
- Make sure that you use references when they're needed and follow academic standards.

In [14]:
## Questions to explore

#1: Visualize the geographic location of the accidents occurances over the years

#2: Visualize the geographic location of the accidents orrances based upon the vehicle type

#3: Create a bar plot to visualize crime occurances from year to year also broken down into types of accidents 

#4: Analyze if there is any seasonality of other patterns correlated with time

#5: Create a prediction tool that can be used to predict the type of accident based upon goegraphic location. 

In [15]:
raw_DF[['CONTRIBUTING FACTOR VEHICLE 1',
 'CONTRIBUTING FACTOR VEHICLE 2',
 'CONTRIBUTING FACTOR VEHICLE 3',
 'CONTRIBUTING FACTOR VEHICLE 4',
 'CONTRIBUTING FACTOR VEHICLE 5',
 'VEHICLE TYPE CODE 1',
 'VEHICLE TYPE CODE 2',
 'VEHICLE TYPE CODE 3',
 'VEHICLE TYPE CODE 4',
 'VEHICLE TYPE CODE 5']]

,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,TAXI,TAXI,NaN,NaN,NaN
1,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,TAXI,NaN,NaN,NaN,NaN
2,Unspecified,NaN,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
3,Unspecified,NaN,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
4,Unspecified,NaN,NaN,NaN,NaN,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
5,Other Vehicular,Following Too Closely,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
6,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
7,Driver Inattention/Distraction,Driver Inattention/Distraction,Unspecified,Unspecified,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN
8,Unspecified,Unspecified,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
9,Driver Inattention/Distraction,Driver Inattention/Distraction,Unspecified,NaN,NaN,TAXI,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN


In [16]:
collections.Counter(list(raw_DF['CONTRIBUTING FACTOR VEHICLE 1']))

Counter({nan: 2873,
         'Accelerator Defective': 244,
         'Aggressive Driving/Road Rage': 1803,
         'Alcohol Involvement': 4782,
         'Animals Action': 306,
         'Backing Unsafely': 18586,
         'Brakes Defective': 1407,
         'Cell Phone (hand-held)': 65,
         'Cell Phone (hands-free)': 133,
         'Driver Inattention/Distraction': 75813,
         'Driver Inexperience': 7403,
         'Driverless/Runaway Vehicle': 41,
         'Drugs (Illegal)': 308,
         'Failure to Keep Right': 1067,
         'Failure to Yield Right-of-Way': 28732,
         'Fatigued/Drowsy': 23260,
         'Fell Asleep': 808,
         'Following Too Closely': 2506,
         'Glare': 1203,
         'Headlights Defective': 45,
         'Illness': 1775,
         'Lane Marking Improper/Inadequate': 253,
         'Lost Consciousness': 13945,
         'Obstruction/Debris': 809,
         'Other Electronic Device': 1551,
         'Other Lighting Defects': 43,
         'Other Vehicula

In [17]:
collections.Counter(list(raw_DF['BOROUGH']))

Counter({'BRONX': 78176,
         'BROOKLYN': 184622,
         'MANHATTAN': 155788,
         'QUEENS': 155957,
         'STATEN ISLAND': 26770})

In [18]:
set(list(raw_DF['BOROUGH']))

{'BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'}